# Proyecto Final

***Equipo 07***

- Aide Jazmín González Cruz
- Elena Villalobos Nolasco
- Carolina Acosta Tovany

#### Instrucciones

El proyecto/examen final consistirá en:

La implementación del algoritmo de filtrado colaborativo utilizando la metodología vista en clase (uso de otra metodología no se calificará).

Todos los algoritmos de aprendizaje de máquina que se utilicen deberán haber sido creados por ustedes. Sólo podrán utilizar Transformers y funciones de apoyo de scikit-learn (para realizar la división de los datos en entrenamiento y prueba, o el procedimiento de validación cruzada, etc.) mas ningún estimator (regresión logística, máquina de vectores de soporte, k medias, etc.). 

Se deberá explicar como se obtuvo la k con la que se generó el resultado final.

Se utilizarán los archivos con el conjunto pequeño de calificaciones y películas ubicado en la siguiente https://www.kaggle.com/rounakbanik/the-movies-dataset:

- **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

Con el fin de mejorar la calificación (opcional, puntos extra), se podrán utilizar los algoritmos desarrollado en las tareas del curso y los datos relevantes (los que hacen match con los datos anteriores) contenidos en los archivos:

- **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

La métrica con la que se determinará el desempeño del algoritmo es el NDCG 

(https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG)

Una vez obtenida la matriz de calificaciones, el programa deberá ser capaz de regresar las 5 mejores recomendaciones del o de los usuarios que se consulten.

El proyecto se entregará en un Jupyter notebook. El readme file debe contener las instrucciones para que se ejecute el código. Deben cerciorarse que siguiendo esas instrucciones el programa corre sin errores. 

Se deberá subir a la carpeta proyecto_final/equipo_xx en el repositorio GitHub antes de las 7:00 am del día del examen final (14 de diciembre de 2020).    

In [90]:
# Importación de paqueterías necesarias
import pandas as pd
import numpy as np
import random

In [27]:
# Importación de datos
links_small = pd.read_csv('links_small.csv')
links_small.head
# Internet Movie Database (IMDb, en español: Base de datos de películas en Internet) 
# es una base de datos en línea que almacena información relacionada con películas, 
# personal de equipo de producción (incluyendo directores y productores), actores, 
# series de televisión, programas de televisión, videojuegos, actores de doblaje y, 
# más recientemente, personajes ficticios que aparecen en los medios de 
# entretenimiento visual. Recibe más de 100 millones de usuarios únicos al mes y 
# cuenta con una versión móvil.4​ IMDb fue inaugurada el 17 de octubre de 1990,2​ 
# y en 1998 fue adquirida por Amazon.com.5​

# The Movie Database(TMDB)es una base de datos gratuita y abierta con millones de 
# datos relacionados con el mundo del cine. Es usada cada mes por miles de 
# aplicaciones que aprovechan su API para mostrar informaciones de todo tipo sobre 
# las más variadas pelí­culas, siendo la fuente de datos de centros tan conocidos 
# como Moovida, XBMC, Plex, MythTV o MediaPortal.

<bound method NDFrame.head of       movieId   imdbId    tmdbId
0           1   114709     862.0
1           2   113497    8844.0
2           3   113228   15602.0
3           4   114885   31357.0
4           5   113041   11862.0
...       ...      ...       ...
9120   162672  3859980  402672.0
9121   163056  4262980  315011.0
9122   163949  2531318  391698.0
9123   164977    27660  137608.0
9124   164979  3447228  410803.0

[9125 rows x 3 columns]>

In [28]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [29]:
def get_repeated_values(df, col, top):
    top_5 = df.groupby([col])[col]\
                    .count()\
                    .sort_values(ascending = False)\
                    .head(3)
    indexes_top_5 = top_5.index
    
    if ((top == 1) and (len(indexes_top_5) > 0)):
        return indexes_top_5[0]
    elif ((top == 2) and (len(indexes_top_5) > 1)):
        return indexes_top_5[1]
    elif ((top == 3) and (len(indexes_top_5) > 2)):
        return indexes_top_5[2]
    else: 
        return 'undefined'
    
def numeric_profiling(df, col):
    """
    Profiling for numeric columns. 
    
    :param: column to analyze
    :return: dictionary
    """
    profiling = {}

    profiling.update({'max': df[col].max(),
                     'min': df[col].min(),
                     'mean': df[col].mean(),
                     'stdv': df[col].std(),
                     '25%': df[col].quantile(.25),
                     'median': df[col].median(),
                     '75%': df[col].quantile(.75),
                     'kurtosis': df[col].kurt(),
                     'skewness': df[col].skew(),
                     'uniques': df[col].nunique(),
                     'prop_missings': df[col].isna().sum()/df.shape[0]*100,
                     'top1_repeated': get_repeated_values(df, col, 1),
                     'top2_repeated': get_repeated_values(df, col, 2),
                     'top3_repeated': get_repeated_values(df, col, 3)})   
    return profiling

def int_transformation(col, df):
    """
    Recibe la columna que hay que transformar a NUMERIC (entera) y el data frame al que pertenece.
    :param: column, dataframe
    :return: column
    """
    return df[col].astype(int)

In [30]:
# variables numéricas
number_variables = links_small.select_dtypes(include = 'number').columns.values
(len(number_variables), number_variables)
numeric_profiling_output = {elem: numeric_profiling(links_small, elem)\
                            for elem in number_variables}
df_numeric_profiling = pd.DataFrame(numeric_profiling_output).reset_index()
df_numeric_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_numeric_profiling

,metric,movieId,imdbId,tmdbId
0,max,164979.000000,5.794766e+06,416437.000000
1,min,1.000000,4.170000e+02,2.000000
2,mean,31123.291836,4.798244e+05,39104.545544
3,stdv,40782.633604,7.431774e+05,62814.519801
4,25%,2850.000000,8.884600e+04,9451.750000
5,median,6290.000000,1.197780e+05,15852.000000
6,75%,56274.000000,4.284410e+05,39160.500000
7,kurtosis,0.375266,7.671923e+00,9.420059
8,skewness,1.259461,2.598949e+00,3.026661
9,uniques,9125.000000,9.125000e+03,9112.000000


In [31]:
ratings_small = pd.read_csv('ratings_small.csv')
ratings_small.head

<bound method NDFrame.head of         userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]>

In [32]:
# variables numéricas
number_variables = ratings_small.select_dtypes(include = 'number').columns.values
(len(number_variables), number_variables)
numeric_profiling_output = {elem: numeric_profiling(ratings_small, elem)\
                            for elem in number_variables}
df_numeric_profiling = pd.DataFrame(numeric_profiling_output).reset_index()
df_numeric_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_numeric_profiling

,metric,userId,movieId,rating,timestamp
0,max,671.000000,163949.000000,5.000000,1.476641e+09
1,min,1.000000,1.000000,0.500000,7.896520e+08
2,mean,347.011310,12548.664363,3.543608,1.129639e+09
3,stdv,195.163838,26369.198969,1.058064,1.916858e+08
4,25%,182.000000,1028.000000,3.000000,9.658478e+08
5,median,367.000000,2406.500000,4.000000,1.110422e+09
6,75%,520.000000,5418.000000,4.000000,1.296192e+09
7,kurtosis,-1.229106,6.664016,0.088397,-1.077084e+00
8,skewness,-0.154514,2.689261,-0.667620,3.216134e-01
9,uniques,671.000000,9066.000000,10.000000,7.814100e+04


In [39]:
# movies_metadata = pd.read_csv('movies_metadata.csv')
# movies_metadata.head

In [9]:
#keywords = pd.read_csv('keywords.csv')
#keywords.head

In [10]:
#credits = pd.read_csv('credits.csv')
#credits.head

In [33]:
# Películas en catálogo que no han calificado los usuarios
df_mov_u = pd.DataFrame(ratings_small['movieId'])
df_mov = pd.DataFrame(links_small['movieId'])

common = df_mov.merge(df_mov_u, on=["movieId"])
result = df_mov[~df_mov.movieId.isin(common.movieId)]
result.shape

(59, 1)

In [34]:
# Construyendo la matriz Y_ai
y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
y_ia = y_ia.reset_index()
y_ia

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7.0,3.0,8.518667e+08
1,1,114709,862.0,9.0,4.0,9.386292e+08
2,1,114709,862.0,13.0,5.0,1.331380e+09
3,1,114709,862.0,15.0,2.0,9.979383e+08
4,1,114709,862.0,19.0,3.0,8.551901e+08
...,...,...,...,...,...,...
100058,162672,3859980,402672.0,611.0,3.0,1.471524e+09
100059,163056,4262980,315011.0,NaN,NaN,NaN
100060,163949,2531318,391698.0,547.0,5.0,1.476419e+09
100061,164977,27660,137608.0,NaN,NaN,NaN


In [35]:
#y_ia.pivot(index="userId", columns="movieId", values="rating") 
y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
y_ia

movieId,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
userId,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
y_ia = pd.DataFrame(y_ia.to_records())
y_ia

,userId,1,2,3,4,5,6,7,8,9,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,668.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,670.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
y_ia = y_ia[pd.notnull(y_ia['userId'])]
y_ia

,userId,1,2,3,4,5,6,7,8,9,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,668.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,670.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
y_ia['userId'] = y_ia['userId'].astype(int)
y_ia

/home/aide/.pyenv/versions/3.7.4/envs/itam_intro_to_ds/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,userId,1,2,3,4,5,6,7,8,9,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
y_ia = y_ia.drop(['userId'], axis=1)
y_ia

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
users, movies = y_ia.shape

In [42]:
y_ia[np.isnan(y_ia)] = 0
y_ia

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

In [64]:
def load_data():
    """
    Carga datos
    Regresa dataframe de Usuarios, Películas y raitings
    """
    
    # Carga de datos
    links_small = pd.read_csv('links_small.csv')
    ratings_small = pd.read_csv('ratings_small.csv')
    
    # Películas en catálogo que no han calificado los usuarios
    df_mov_u = pd.DataFrame(ratings_small['movieId'])
    df_mov = pd.DataFrame(links_small['movieId'])

    common = df_mov.merge(df_mov_u, on=["movieId"])
    result = df_mov[~df_mov.movieId.isin(common.movieId)]
    
    # Construyendo la matriz Y_ai
    y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
    y_ia = y_ia.reset_index()
    #y_ia.pivot(index="userId", columns="movieId", values="rating") 
    y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
    y_ia = pd.DataFrame(y_ia.to_records())
    # Eliminando usuario Nan
    y_ia = y_ia[pd.notnull(y_ia['userId'])]
    # Borrando columna 1 con user_id
    y_ia = y_ia.drop(['userId'], axis=1)
    # Cambiando Nan por zeros
    #y_ia[np.isnan(y_ia)] = 0
    
    return y_ia
    

In [84]:
Y = load_data()
Y

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# Cambiando Nan por zeros
Y_0 = Y.copy()
Y_0[np.isnan(Y_0)] = 0
Y_0

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Cambiando Nan a boleanos
Y.isna()

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
5,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
668,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
669,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
670,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [87]:
Prueba = Y.iloc[[0,1,2,3,4], : 10]
Prueba

,1,2,3,4,5,6,7,8,9,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
Prueba.iloc[1][1] = 1
Prueba.iloc[1][4] = 5
Prueba

,1,2,3,4,5,6,7,8,9,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# Fijando semilla
random.seed(0)
random_matrix = np.random.randint(1,5,(5,10))
random_matrix = pd.DataFrame(random_matrix)
random_matrix

,0,1,2,3,4,5,6,7,8,9
0,3,2,3,1,1,1,4,3,2,4
1,2,2,4,2,3,3,2,4,2,2
2,2,1,1,4,4,4,2,3,2,1
3,1,1,3,3,2,4,2,3,4,2
4,3,3,2,3,4,4,2,1,1,4


In [93]:
Prueba_bool = Prueba.isna()
Prueba_bool

,1,2,3,4,5,6,7,8,9,10
1,True,True,True,True,True,True,True,True,True,True
2,True,False,True,True,False,True,True,True,True,False
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,False
5,True,True,False,True,True,True,True,True,True,True


In [104]:
random_matrix.iloc[1][1]*Prueba_bool.iloc[1][1]
random_matrix.iloc[1]

0    2
1    2
2    4
3    2
4    3
5    3
6    2
7    4
8    2
9    2
Name: 1, dtype: int64

In [123]:
nvos = []
for i in range (len(random_matrix.iloc[1])):
    nvos.append([i+1,random_matrix.iloc[1][i]*Prueba_bool.iloc[1][i]])
    
nvos = pd.DataFrame(nvos)
nvos.columns = ['id_movie','rating_recom']
nvos = nvos.sort_values(by=['rating_recom'], ascending=False)
nvos

,id_movie,rating_recom
2,3,4
7,8,4
5,6,3
0,1,2
3,4,2
6,7,2
8,9,2
1,2,0
4,5,0
9,10,0


In [133]:
nvos['id_movie'].head(5).to_numpy()

array([3, 8, 6, 1, 4])

In [126]:
recomendaciones = nvos.iloc[[0,1,2,3,4], : 1].to_numpy()
recomendaciones

array([[3],
       [8],
       [6],
       [1],
       [4]])

In [129]:
np.concatenate(recomendaciones, axis=0 )

array([3, 8, 6, 1, 4])

In [147]:
def recomendaciones_id(Y, MCompleta, id_user, top=5):
    """
    Regresa id de las peliculas reomendadas dado un usuario
    Param: Recibe matriz con Nan, id del usurario y el top por default 5
    Return: Arreglo de ids de las películas sugeridas
    """
    # Matriz auxiliar booleana
    Y_bool = Y.isna()
    # Se multiplica la matriz boleana con Y para poner en cero los ratings dados por el usuario
    nvos = []
    #print(MCompleta.iloc[id_user-1])
    for i in range (len(MCompleta.iloc[id_user-1])):
        #Se suma 1 por que los usuarios inician en 1
        nvos.append([i+1,MCompleta.iloc[id_user-1][i]*Y_bool.iloc[id_user-1][i]])
        
    # nvos es un arreglo se pasa a dataframe para mejor manejabilidad
    nvos = pd.DataFrame(nvos)
    # se colocan nombres a las columnas
    nvos.columns = ['id_movie','rating_recom']
    # se ordenan de forma descendente
    nvos = nvos.sort_values(by=['rating_recom'], ascending=False)
    # Obteniendo el top
    recomendaciones = nvos['id_movie'].head(5).to_numpy()
    
    return recomendaciones


In [148]:
recomendaciones_id(Prueba, random_matrix, 2)

array([3, 8, 6, 1, 4])

Tenemos una $k = 1$

In [47]:
# Fijando semilla
random.seed(0)
# Creando el vector V que son las películas al azar
V = np.random.randint(1,9,size = (1,movies))
V.shape

(1, 9125)

In [48]:
# Sacando el vector U que son los usuarios
U = np.random.randint(1,9,size = (1,users))
U.shape

(1, 671)

In [2]:
from sympy import *

In [5]:
Y = pd.DataFrame([[5,0,7],[1,2,0]])
Y = Matrix(Y)
Y

Matrix([
[5, 0, 7],
[1, 2, 0]])

In [6]:
r,c = Y.shape
print("rows: ", r, " cols: ", c)

rows:  2  cols:  3


In [35]:
u, v = symbols('u v')

In [36]:
U = ones(r,1)*u
U

Matrix([
[u],
[u]])

In [37]:
V = ones(1,c)*v
V

Matrix([[v, v, v]])

In [38]:
Vaux = Matrix([[2,7,8]])
Vaux

Matrix([[2, 7, 8]])

In [39]:
Exp = U*Vaux
Exp

Matrix([
[2*u, 7*u, 8*u],
[2*u, 7*u, 8*u]])

In [40]:
# Obteniendo vector U (fijando V)
Uaux = []
# Rows
for i in range(r):
    e = 0
    # Columns
    for j in range(c):
        print("Y ",Y[i,j])
        print("E ",Exp[i,j])
        print("YE ",Y[i,j]-Exp[i,j])
        if ((Y[i,j]) != 0):
            e = e + ((Y[i,j]-Exp[i,j])**2)/2
            print("e: ",e)
        
    e = e + (u**2)/2     
    e
    e = derive_by_array(e, u)
    e
    h = solve(e,u)
    Uaux.append(h)
    print("sol ",h)
    
print(Uaux)

Y  5
E  2*u
YE  5 - 2*u
e:  (5 - 2*u)**2/2
Y  0
E  7*u
YE  -7*u
Y  7
E  8*u
YE  7 - 8*u
e:  (5 - 2*u)**2/2 + (7 - 8*u)**2/2
sol  [22/23]
Y  1
E  2*u
YE  1 - 2*u
e:  (1 - 2*u)**2/2
Y  2
E  7*u
YE  2 - 7*u
e:  (1 - 2*u)**2/2 + (2 - 7*u)**2/2
Y  0
E  8*u
YE  -8*u
sol  [8/27]
[[22/23], [8/27]]


In [14]:
Uaux = Matrix(Uaux)
Uaux

Matrix([
[22/23],
[ 8/27]])

In [15]:
Exp = Uaux*V
Exp

Matrix([
[22*v/23, 22*v/23, 22*v/23],
[ 8*v/27,  8*v/27,  8*v/27]])

In [16]:
# Obteniendo vector V (fijando U)
Vaux = []

# Columns
for j in range(c):
    e = 0
    # Rows
    for i in range(r):    
        print("Y ",Y[i,j])
        print("E ",Exp[i,j])
        print("YE ",Y[i,j]-Exp[i,j])
        if ((Y[i,j]) != 0):
            e = e + ((Y[i,j]-Exp[i,j])**2)/2
            
        print("e: ",e)
        
    e = e + (v**2)/2     
    e
    e = derive_by_array(e, v)
    e
    h = solve(e,v)
    Vaux.append(h)
    print("sol ",h)
    
print(Vaux)

Y  5
E  22*v/23
YE  5 - 22*v/23
e:  (5 - 22*v/23)**2/2
Y  1
E  8*v/27
YE  1 - 8*v/27
e:  (1 - 8*v/27)**2/2 + (5 - 22*v/23)**2/2
sol  [1958634/772333]
Y  0
E  22*v/23
YE  -22*v/23
e:  0
Y  2
E  8*v/27
YE  2 - 8*v/27
e:  (2 - 8*v/27)**2/2
sol  [432/793]
Y  7
E  22*v/23
YE  7 - 22*v/23
e:  (7 - 22*v/23)**2/2
Y  0
E  8*v/27
YE  -8*v/27
e:  (7 - 22*v/23)**2/2
sol  [3542/1013]
[[1958634/772333], [432/793], [3542/1013]]


In [17]:
Vaux = Matrix(Vaux)
Vaux

Matrix([
[1958634/772333],
[       432/793],
[     3542/1013]])

In [18]:
Uaux

Matrix([
[22/23],
[ 8/27]])

In [26]:
sum(Uaux)

778/621

In [19]:
Uaux*Vaux.T

Matrix([
[1873476/772333, 9504/18239,   3388/1013],
[ 580336/772333,    128/793, 28336/27351]])

In [20]:
Y

Matrix([
[5, 0, 7],
[1, 2, 0]])

In [21]:
Y-(Uaux*Vaux.T)

Matrix([
[1988189/772333, -9504/18239,    3703/1013],
[ 191997/772333,    1458/793, -28336/27351]])

In [22]:
R = (Y-(Uaux*Vaux.T))
R

Matrix([
[1988189/772333, -9504/18239,    3703/1013],
[ 191997/772333,    1458/793, -28336/27351]])

In [23]:
suma = 0
for i in range(r):
    for j in range(c):
        suma = suma + (Y[i,j]-R[i,j])**2
        
suma

2821224774022858310335052715808/148442291175701261271076965369

In [24]:
suma/2

1410612387011429155167526357904/148442291175701261271076965369

In [25]:
(suma/2).evalf()

9.50276619849381

In [57]:
def get_vect_u(Exp,Y):
    # Obteniendo vector U (fijando V)
    Uaux = []
    # Rows
    for i in range(r):
        e = 0
        # Columns
        for j in range(c):
            print("Y ",Y[i,j])
            print("E ",Exp[i,j])
            print("YE ",Y[i,j]-Exp[i,j])
            if ((Y[i,j]) != 0):
                e = e + ((Y[i,j]-Exp[i,j])**2)/2
                #print("e: ",e)

        e = e + (u**2)/2     
        e
        e = derive_by_array(e, u)
        e
        h = solve(e,u)
        Uaux.append(h)
        print("sol ",h)

    print(Uaux)
    
    return Matrix(Uaux)

def get_vect_v(Exp,Y):
    # Obteniendo vector V (fijando U)
    Vaux = []

    # Columns
    for j in range(c):
        e = 0
        # Rows
        for i in range(r):    
            print("Y ",Y[i,j])
            print("E ",Exp[i,j])
            print("YE ",Y[i,j]-Exp[i,j])
            if ((Y[i,j]) != 0):
                e = e + ((Y[i,j]-Exp[i,j])**2)/2

            print("e: ",e)

        e = e + (v**2)/2     
        e
        e = derive_by_array(e, v)
        e
        h = solve(e,v)
        Vaux.append(h)
        print("sol ",h)

    print(Vaux)
    
    return Matrix(Vaux)

def suma_r_uv(R,Y):
    suma = 0
    for i in range(r):
        for j in range(c):
            suma = suma + (Y[i,j]-R[i,j])**2

    return((suma/2).evalf())


def factorizacion_mat(Y):
    costo = 10
    ciclos = 2
    r,c = Y.shape
    u, v = symbols('u v')
    U = ones(r,1)*u
    V = ones(1,c)*v
    # Iniciar de manera aleatoria vector Vaux
    Vaux = Matrix([[2],[7],[8]])
    i = 3
    
    #while costo > 10:
    for k in range(i):
        Exp = U*Vaux.T
        Uaux = get_vect_u(Exp,Y)
        Exp = Uaux*V
        Vaux = get_vect_v(Exp,Y)
        
        R = Y-(Uaux*Vaux.T)
        print("U: ",Uaux,"V:",Vaux)
        print("SR: ", suma_r_uv(R,Y),"SU: ", sum(Uaux).evalf(),"SV: ", sum(Vaux).evalf())
        costo = suma_r_uv(R,Y) + sum(Uaux).evalf() + sum(Vaux).evalf()
        print(costo)
        
    return R
        
    
    
    

In [58]:
factorizacion_mat(Y)

Y  5
E  2*u
YE  5 - 2*u
Y  0
E  7*u
YE  -7*u
Y  7
E  8*u
YE  7 - 8*u
sol  [22/23]
Y  1
E  2*u
YE  1 - 2*u
Y  2
E  7*u
YE  2 - 7*u
Y  0
E  8*u
YE  -8*u
sol  [8/27]
[[22/23], [8/27]]
Y  5
E  22*v/23
YE  5 - 22*v/23
e:  (5 - 22*v/23)**2/2
Y  1
E  8*v/27
YE  1 - 8*v/27
e:  (1 - 8*v/27)**2/2 + (5 - 22*v/23)**2/2
sol  [1958634/772333]
Y  0
E  22*v/23
YE  -22*v/23
e:  0
Y  2
E  8*v/27
YE  2 - 8*v/27
e:  (2 - 8*v/27)**2/2
sol  [432/793]
Y  7
E  22*v/23
YE  7 - 22*v/23
e:  (7 - 22*v/23)**2/2
Y  0
E  8*v/27
YE  -8*v/27
e:  (7 - 22*v/23)**2/2
sol  [3542/1013]
[[1958634/772333], [432/793], [3542/1013]]
U:  Matrix([[22/23], [8/27]]) V: Matrix([[1958634/772333], [432/793], [3542/1013]])
SR:  9.50276619849381 SU:  1.25281803542673 SV:  6.57730840338357
17.3328926373041
Y  5
E  1958634*u/772333
YE  5 - 1958634*u/772333
Y  0
E  432*u/793
YE  -432*u/793
Y  7
E  3542*u/1013
YE  7 - 3542*u/1013
sol  [22743362352710422348/12032272356064417001]
Y  1
E  1958634*u/772333
YE  1 - 1958634*u/772333
Y  2
E  432*u

Matrix([
[ 803745785458341843296128651313835307363211104934674262754505960417957749937354204174013436310306495925949163268415313206205816297404863090294745871944870655381202591953611511995169235703110800792829283558894894490492670837338643657005190483288294950230486992894983363236750837023007202300630973526854459170570373396707552287458671007877604510040634869627148282980442995398953249116301894471238566774962998274456456194495566853191243801420688495211455441545856797030806676166651006421335800897670100377184496629127817210427293480656909142968441420545441470482016422734031799101043763064257722659068292929942777316968952975014800981008625683275898792634909084545341218525968662141806458695051247776104961495620873061255999081565123995234234982993015809365933637441864947163332660374253407930320969039980695353579555271803862061388128433178079163704969118373786163762087772907190125/9335672528585261255182105651379023822657982672970801998294681744596354730118915586466260242431754758803825522

In [48]:
Vaux

Matrix([[2, 7, 8]])